In [1]:
import re
import time
import json
import os
from utils import FireCrawl, crawl_2_url
from datetime import datetime, timezone
from pymongo import MongoClient



TIME_RUN = datetime.now(timezone.utc).strftime('%Y-%m-%d-%H-%M-%S')
TIME_NOW = datetime.now(timezone.utc)
WHO_URL_LIST = [
    'https://data.who.int/dashboards/covid19/summary',
    'https://data.who.int/dashboards/covid19/circulation',
    'https://data.who.int/dashboards/covid19/cases',
    'https://data.who.int/dashboards/covid19/deaths',
    'https://data.who.int/dashboards/covid19/hospitalizations',
    'https://data.who.int/dashboards/covid19/vaccines',
    'https://data.who.int/dashboards/covid19/variants'
]
who_summary_soup = FireCrawl(WHO_URL_LIST[0]).crawl()
# time.sleep(30)
# who_cirulation_soup = FireCrawl(WHO_URL_LIST[1]).crawl()
# time.sleep(30)
# who_cases_soup = FireCrawl(WHO_URL_LIST[2]).crawl()
# time.sleep(30)
# who_deaths_soup = FireCrawl(WHO_URL_LIST[3]).crawl()
# time.sleep(30)
# who_hospitalizations_soup = FireCrawl(WHO_URL_LIST[4]).crawl()
# time.sleep(30)
# who_vaccines_soup = FireCrawl(WHO_URL_LIST[5]).crawl()
# time.sleep(30)
# who_variants_soup = FireCrawl(WHO_URL_LIST[6]).crawl()



[05/15/25 10:52:25] INFO     Submitted: data.who.int/dashboards/covid19/summary;                        ]8;id=455792;file:///data/ykx/epi-crawl/utils.py\utils.py]8;;\:]8;id=595457;file:///data/ykx/epi-crawl/utils.py#55\55]8;;\

[05/15/25 10:52:26] INFO     Scraping: data.who.int/dashboards/covid19/summary;                         ]8;id=732155;file:///data/ykx/epi-crawl/utils.py\utils.py]8;;\:]8;id=71050;file:///data/ykx/epi-crawl/utils.py#72\72]8;;\

[05/15/25 10:52:38] INFO     Completed: data.who.int/dashboards/covid19/summary;                        ]8;id=722000;file:///data/ykx/epi-crawl/utils.py\utils.py]8;;\:]8;id=222604;file:///data/ykx/epi-crawl/utils.py#69\69]8;;\

In [ ]:
# 0. summary
p1_soup = who_summary_soup.find_all('div', attrs={'id': 'PageContent_C493_Col00', 'class': 'sf_colsIn col-md-12'})[0]
p2_soup = who_summary_soup.find_all('div', attrs={'id': 'PageContent_C629_Col00', 'class': 'sf_colsIn col-md-9'})[0]
WHO_realtime_summary = [
    {
        'date': str(TIME_NOW.replace(tzinfo=timezone.utc)),
        'summary_circulation': p1_soup.find_all('p')[0].text,
        'summary_cases': p1_soup.find_all('p')[10].text,
        'summary_variant': p1_soup.find_all('div', class_ = 'sfContentBlock sf-Long-text')[9].text,
        'summary_severity': p2_soup.find_all('p')[0].text,
        'summary_death': p2_soup.find_all('p')[10].text
    }
]
WHO_realtime_summary


[{'date': '2025-05-15 02:52:24.891788+00:00',
  'summary_circulation': 'In the week of 21 to 27 April 2025, a total of 49,320 samples were tested for SARS-CoV-2, the virus that causes COVID-19, across 76 countries. WHO collects this data from a global network of sentinel and systematic virologic surveillance sites. Out of these, 1,604 (3.2%) samples tested positive for the virus (Table 1.1 and 1.2). During this period, overall SARS-CoV-2 activity was low. Compared to the previous seven days, most reporting countries showed a stable trend. An increasing trend or elevated activity was reported from a few countries in Western Africa, Central America and Caribbean, Eastern Europe, South West Europe, and Western Asia.',
  'summary_cases': 'In the 28-day period from 31 March 2025 to 27 April 2025, 89 countries across five WHO regions reported new COVID-19 cases. During this 28-day period, a total of 25,463 new cases were reported, which is a decrease compared to the 59,024 new cases reported

('In the 28-day period from 31 March 2025 to 27 April 2025, 38 countries across four WHO regions reported new COVID-19 deaths. During this 28-day period, a total of 1,458 new deaths were reported, which is a decrease compared to the 2,350 new deaths reported from 43 countries in the previous 28-day period (Table 5.1 and 5.2). Seven countries from the Americas and Europe showed an increase in new deaths of over 10%. In March 2025, 88% of reported deaths with age information occurred in the population aged 65 and over.',)